In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [2]:
dataTrain=pd.read_csv("C:/Users/abira/Downloads/allAtt_onehot_large_train.csv")
dataTest=pd.read_csv("C:/Users/abira/Downloads/allAtt_onehot_large_test.csv")
print(dataTrain.head())
print(dataTrain.shape)

   FTR  HTGS  ATGS  HTGC  ATGC  HTP  ATP  HM1  HM2  HM3  ...  ATWinStreak5  \
0    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
1    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
2    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
3    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
4    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   

   ATLossStreak3  ATLossStreak5  HTGD  ATGD  DiffPts  DiffFormPts  DiffLP  \
0              0              0   0.0   0.0      0.0          0.0   -11.0   
1              0              0   0.0   0.0      0.0          0.0     2.0   
2              0              0   0.0   0.0      0.0          0.0     2.0   
3              0              0   0.0   0.0      0.0          0.0   -17.0   
4              0              0   0.0   0.0      0.0          0.0     4.0   

   final1  final2  
0     1.0     0.0  
1     1.0     0.0  
2     0.

In [3]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 1).
# Each input sequence will be of size (28, 1).
input_dim = 27

units = 64
output_size = 2  # labels are from Win or Loss

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(input_dim,1))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(input_dim,1)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [4]:
x_train, y_train = dataTrain.iloc[:,1:28].values,dataTrain.iloc[:,28:].values
x_train=np.reshape(x_train,(1860,27,1))
x_test, y_test = dataTest.iloc[:,1:28].values,dataTest.iloc[:,28:].values
x_test=np.reshape(x_test,(800,27,1))

In [5]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer="Adam",
    metrics=["categorical_accuracy"],
)

model.summary()

C:\Users\abira\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,282 (67.51 KB)

 Trainable params: 17,154 (67.01 KB)

 Non-trainable params: 128 (512.00 B)

In [6]:
model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=10
)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - categorical_accuracy: 0.6097 - loss: 0.7178 - val_categorical_accuracy: 0.5263 - val_loss: 0.6903
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - categorical_accuracy: 0.6701 - loss: 0.6055 - val_categorical_accuracy: 0.5263 - val_loss: 0.6828
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - categorical_accuracy: 0.6630 - loss: 0.6066 - val_categorical_accuracy: 0.5263 - val_loss: 0.6827
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - categorical_accuracy: 0.6681 - loss: 0.6049 - val_categorical_accuracy: 0.5263 - val_loss: 0.6774
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - categorical_accuracy: 0.6625 - loss: 0.6153 - val_categorical_accuracy: 0.5875 - val_loss: 0.6595
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - categorical_accuracy: 0.6746 - loss: 0.6002 - val_categorical_accuracy: 0.5663 - val_loss: 0.6664
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - categorical_accuracy: 0.6932 - loss: 0.59

In [7]:
model.predict(np.expand_dims(x_test[0], 0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step


array([[-0.03681858, -0.18077762]], dtype=float32)